Okay, let's continue to improve our prediction. Here, we'll be focuing on looking on the numerical and categorical column.

In [27]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
train = pd.read_csv("Datasets/train.csv")
test = pd.read_csv("Datasets/test.csv")

In [3]:
# Gets the categorical and numerical columns
categorical = [c for c in train.columns 
               if train.dtypes[c] == 'object']

numerical = [c for c in train.columns 
             if train.dtypes[c] != 'object']

# Removes the Id and prediction target
numerical.remove('Id')
numerical.remove('SalePrice')

In [4]:
len(numerical), len(categorical)

(36, 43)

In [5]:
print(categorical)
print(numerical)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'Enclos

In [6]:
train[categorical].head()

MSZoning Street Alley LotShape LandContour Utilities LotConfig LandSlope  \
0       RL   Pave   NaN      Reg         Lvl    AllPub    Inside       Gtl   
1       RL   Pave   NaN      Reg         Lvl    AllPub       FR2       Gtl   
2       RL   Pave   NaN      IR1         Lvl    AllPub    Inside       Gtl   
3       RL   Pave   NaN      IR1         Lvl    AllPub    Corner       Gtl   
4       RL   Pave   NaN      IR1         Lvl    AllPub       FR2       Gtl   

  Neighborhood Condition1 Condition2 BldgType HouseStyle RoofStyle RoofMatl  \
0      CollgCr       Norm       Norm     1Fam     2Story     Gable  CompShg   
1      Veenker      Feedr       Norm     1Fam     1Story     Gable  CompShg   
2      CollgCr       Norm       Norm     1Fam     2Story     Gable  CompShg   
3      Crawfor       Norm       Norm     1Fam     2Story     Gable  CompShg   
4      NoRidge       Norm       Norm     1Fam     2Story     Gable  CompShg   

  Exterior1st Exterior2nd MasVnrType ExterQual ExterCond Foundation BsmtQual  \
0     VinylSd     VinylSd    BrkFace        Gd        TA      PConc       Gd   
1     MetalSd     MetalSd       None        TA        TA     CBlock       Gd   
2     VinylSd     VinylSd    BrkFace        Gd        TA      PConc       Gd   
3     Wd Sdng     Wd Shng       None        TA        TA     BrkTil       TA   
4     VinylSd     VinylSd    BrkFace        Gd        TA      PConc       Gd   

  BsmtCond BsmtExposure BsmtFinType1 BsmtFinType2 Heating HeatingQC  \
0       TA           No          GLQ          Unf    GasA        Ex   
1       TA           Gd          ALQ          Unf    GasA        Ex   
2       TA           Mn          GLQ          Unf    GasA        Ex   
3       Gd           No          ALQ          Unf    GasA        Gd   
4       TA           Av          GLQ          Unf    GasA        Ex   

  CentralAir Electrical KitchenQual Functional FireplaceQu GarageType  \
0          Y      SBrkr          Gd        Typ         NaN     Attchd   
1          Y      SBrkr          TA        Typ          TA     Attchd   
2          Y      SBrkr          Gd        Typ          TA     Attchd   
3          Y      SBrkr          Gd        Typ          Gd     Detchd   
4          Y      SBrkr          Gd        Typ          TA     Attchd   

  GarageFinish GarageQual GarageCond PavedDrive PoolQC Fence MiscFeature  \
0          RFn         TA         TA          Y    NaN   NaN         NaN   
1          RFn         TA         TA          Y    NaN   NaN         NaN   
2          RFn         TA         TA          Y    NaN   NaN         NaN   
3          Unf         TA         TA          Y    NaN   NaN         NaN   
4          RFn         TA         TA          Y    NaN   NaN         NaN   

  SaleType SaleCondition  
0       WD        Normal  
1       WD        Normal  
2       WD        Normal  
3       WD       Abnorml  
4       WD        Normal

In [7]:
train[numerical].head()

MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0          60         65.0     8450            7            5       2003   
1          20         80.0     9600            6            8       1976   
2          60         68.0    11250            7            5       2001   
3          70         60.0     9550            7            5       1915   
4          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          2003       196.0         706           0        150          856   
1          1976         0.0         978           0        284         1262   
2          2002       162.0         486           0        434          920   
3          1970         0.0         216           0        540          756   
4          2000       350.0         655           0        490         1145   

   1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  \
0       856       854             0       1710             1             0   
1      1262         0             0       1262             0             1   
2       920       866             0       1786             1             0   
3       961       756             0       1717             1             0   
4      1145      1053             0       2198             1             0   

   FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  TotRmsAbvGrd  Fireplaces  \
0         2         1             3             1             8           0   
1         2         0             3             1             6           1   
2         2         1             3             1             6           1   
3         1         0             3             1             7           1   
4         2         1             4             1             9           1   

   GarageYrBlt  GarageCars  GarageArea  WoodDeckSF  OpenPorchSF  \
0       2003.0           2         548           0           61   
1       1976.0           2         460         298            0   
2       2001.0           2         608           0           42   
3       1998.0           3         642           0           35   
4       2000.0           3         836         192           84   

   EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold  
0              0          0            0         0        0       2    2008  
1              0          0            0         0        0       5    2007  
2              0          0            0         0        0       9    2008  
3            272          0            0         0        0       2    2006  
4              0          0            0         0        0      12    2008

Notice that MSSubClass should be categorical

In [8]:
to_cat = ['MSSubClass']
for col in to_cat:
    numerical.remove(col)
    categorical.append(col)

Let's do ordinal encoding to some columns where order matters!

In [9]:
ordinal_map = {
    "ExterQual": [np.nan,'Po','Fa','TA','Gd','Ex'], 
    "ExterCond": [np.nan,'Po','Fa','TA','Gd','Ex'], 
    "BsmtQual": [np.nan,'Po','Fa','TA','Gd','Ex'], 
    "BsmtCond": [np.nan,'Po','Fa','TA','Gd','Ex'],
    "FireplaceQu": [np.nan,'Po','Fa','TA','Gd','Ex'], 
    "KitchenQual": [np.nan,'Po','Fa','TA','Gd','Ex'], 
    "GarageQual": [np.nan,'Po','Fa','TA','Gd','Ex'], 
    "GarageCond": [np.nan,'Po','Fa','TA','Gd','Ex'],
    "PoolQC": [np.nan,'Po','Fa','TA','Gd','Ex'], 
    "LotShape": [np.nan,'Reg','IR1' ,'IR2','IR3'], 
    "BsmtExposure": [np.nan,'No','Mn','Av','Gd'], 
    "BsmtFinType1": [np.nan,'Unf','LwQ', 'Rec','BLQ','ALQ' , 'GLQ' ], 
    "BsmtFinType2": [np.nan,'Unf','LwQ', 'Rec','BLQ','ALQ' , 'GLQ' ], 
    "HeatingQC": [np.nan,'Po','Fa','TA','Gd','Ex'], 
    "Functional": [np.nan,'Sev','Maj2','Maj1','Mod','Min2','Min1','Typ'], 
    "GarageFinish": [np.nan,'Unf','RFn','Fin'], 
    "Fence": [np.nan,'MnWw','GdWo','MnPrv','GdPrv'],
    "SaleCondition": [np.nan, 'Normal', 'Abnorml', 'AdjLand', 'Alloca', 'Family', 'Partial'],
}

ordinal = [col for col in ordinal_map.keys()]
onehot = [col for col in categorical if col not in ordinal_map.keys()]

In [10]:
print(ordinal)
print(onehot)

['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'FireplaceQu', 'KitchenQual', 'GarageQual', 'GarageCond', 'PoolQC', 'LotShape', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'Functional', 'GarageFinish', 'Fence', 'SaleCondition']
['MSZoning', 'Street', 'Alley', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive', 'MiscFeature', 'SaleType', 'MSSubClass']


In [11]:
numericalp = numerical.copy()
numericalp = ['SalePrice'] + numericalp

In [13]:
corr = train[numericalp].corr()
plt.figure(figsize=(30, 30))
sns.heatmap(corr, xticklabels=1, yticklabels=1, annot=True)

<AxesSubplot:>

Looking closer at correlations

In [14]:
high_corr_pairs = []
thres_pos = 0.8
thres_neg = -0.3

# Assuming 'corr' is a pandas DataFrame containing correlation values
for col in corr:
    # Extract columns with correlations above thres_pos or below thres_neg
    correlated_cols = corr.index[(corr[col] > thres_pos) | (corr[col] < thres_neg)]
    
    for correlated_col in correlated_cols:
        if correlated_col != col:
            correlation_value = corr.at[col, correlated_col]  # Access correlation value directly from the DataFrame
            high_corr_pairs.append((col, correlated_col, correlation_value))

# Separate positive and negative correlation pairs
pairs_pos, pairs_neg = [], []
unique_pairs = set()

for pair in high_corr_pairs:
    col, ccol, correlation_value = pair
    inverse_pair = (ccol, col, correlation_value)
    
    # Check for uniqueness to avoid duplicates
    if pair not in unique_pairs and inverse_pair not in unique_pairs:
        unique_pairs.add(pair)
        
        # Categorize pairs into positive and negative correlations
        if correlation_value > 0:
            pairs_pos.append(pair)
        else:
            pairs_neg.append(pair)

pairs = pairs_pos + pairs_neg     
print(f"Pair Numerical Columns Highest and Lowest Correlations (treshold is {thres_pos} and {thres_neg})\n")   
for pair in pairs:
    col, ccol, correlation_value = pair
    print(f'{col:<13} {ccol:<14} {correlation_value}')


Pair Numerical Columns Highest and Lowest Correlations (treshold is 0.8 and -0.3)

YearBuilt     GarageYrBlt    0.8256674841743408
TotalBsmtSF   1stFlrSF       0.8195299750050339
GrLivArea     TotRmsAbvGrd   0.8254893743088425
GarageCars    GarageArea     0.882475414281462
OverallCond   YearBuilt      -0.37598319560698945
OverallCond   GarageYrBlt    -0.32429673252939795
YearBuilt     EnclosedPorch  -0.3872677829280785
BsmtFinSF1    BsmtUnfSF      -0.49525146925701125
BsmtUnfSF     BsmtFullBath   -0.42290047738321335


In [15]:
print("Correlations with Target Column\n")
corr = train[numericalp].corr()

thres_pos = 0.5
thres_neg = -0.1

neg, neut, pos = [], [], []
for i in range(len(corr['SalePrice'])):
    cor = corr['SalePrice'][i]
    if cor > thres_pos:
        pos.append([cor, corr['SalePrice'].index[i]])
    elif cor < thres_neg:
        neg.append([cor, corr['SalePrice'].index[i]])
    else:
        neut.append([cor, corr['SalePrice'].index[i]])

print(f"Positive Correlations (treshold is {thres_pos})")
for cor in pos:
    score, col = cor
    if col == 'SalePrice': continue
    print(f'{col:<15}: {score:.3f}')

print(f"\nNegative Correlations (treshold is {thres_neg})")
for cor in neg:
    score, col = cor
    print(f'{col:<15}: {score:.3f}')

Correlations with Target Column

Positive Correlations (treshold is 0.5)
OverallQual    : 0.791
YearBuilt      : 0.523
YearRemodAdd   : 0.507
TotalBsmtSF    : 0.614
1stFlrSF       : 0.606
GrLivArea      : 0.709
FullBath       : 0.561
TotRmsAbvGrd   : 0.534
GarageCars     : 0.640
GarageArea     : 0.623

Negative Correlations (treshold is -0.1)
KitchenAbvGr   : -0.136
EnclosedPorch  : -0.129


In [16]:
for col in train.columns:
    if col == 'SalePrice' or col == 'LotFrontage' or col == 'Id': continue
    if col not in numerical and col not in categorical:
        print("Columns to be dropped", col)
        train = train.drop(columns=col)
        test = test.drop(columns=col)

In [17]:
X = train.drop(columns='SalePrice')
X_test = test
y_train = train["SalePrice"]
X_concat = pd.concat([X, X_test])
X_concat
# Notice the number of rows in X_concat, it's concatinating train and test

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0        1          60       RL         65.0     8450   Pave   NaN      Reg   
1        2          20       RL         80.0     9600   Pave   NaN      Reg   
2        3          60       RL         68.0    11250   Pave   NaN      IR1   
3        4          70       RL         60.0     9550   Pave   NaN      IR1   
4        5          60       RL         84.0    14260   Pave   NaN      IR1   
...    ...         ...      ...          ...      ...    ...   ...      ...   
1454  2915         160       RM         21.0     1936   Pave   NaN      Reg   
1455  2916         160       RM         21.0     1894   Pave   NaN      Reg   
1456  2917          20       RL        160.0    20000   Pave   NaN      Reg   
1457  2918          85       RL         62.0    10441   Pave   NaN      Reg   
1458  2919          60       RL         74.0     9627   Pave   NaN      Reg   

     LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0            Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1            Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2            Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3            Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4            Lvl    AllPub       FR2       Gtl      NoRidge       Norm   
...          ...       ...       ...       ...          ...        ...   
1454         Lvl    AllPub    Inside       Gtl      MeadowV       Norm   
1455         Lvl    AllPub    Inside       Gtl      MeadowV       Norm   
1456         Lvl    AllPub    Inside       Gtl      Mitchel       Norm   
1457         Lvl    AllPub    Inside       Gtl      Mitchel       Norm   
1458         Lvl    AllPub    Inside       Mod      Mitchel       Norm   

     Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0          Norm     1Fam     2Story            7            5       2003   
1          Norm     1Fam     1Story            6            8       1976   
2          Norm     1Fam     2Story            7            5       2001   
3          Norm     1Fam     2Story            7            5       1915   
4          Norm     1Fam     2Story            8            5       2000   
...         ...      ...        ...          ...          ...        ...   
1454       Norm    Twnhs     2Story            4            7       1970   
1455       Norm   TwnhsE     2Story            4            5       1970   
1456       Norm     1Fam     1Story            5            7       1960   
1457       Norm     1Fam     SFoyer            5            5       1992   
1458       Norm     1Fam     2Story            7            5       1993   

      YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0             2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1             1976     Gable  CompShg     MetalSd     MetalSd       None   
2             2002     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3             1970     Gable  CompShg     Wd Sdng     Wd Shng       None   
4             2000     Gable  CompShg     VinylSd     VinylSd    BrkFace   
...            ...       ...      ...         ...         ...        ...   
1454          1970     Gable  CompShg     CemntBd     CmentBd       None   
1455          1970     Gable  CompShg     CemntBd     CmentBd       None   
1456          1996     Gable  CompShg     VinylSd     VinylSd       None   
1457          1992     Gable  CompShg     HdBoard     Wd Shng       None   
1458          1994     Gable  CompShg     HdBoard     HdBoard    BrkFace   

      MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond  \
0          196.0        Gd        TA      PConc       Gd       TA   
1            0.0        TA        TA     CBlock       Gd       TA   
2          162.0        Gd        TA      PConc       Gd       TA   
3            0.0        TA        TA     BrkTil       TA       Gd   

Use pipeline for clean code

In [18]:
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(fill_value=0)),
    ('scaler', StandardScaler())
])

frontage_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean', add_indicator=True)),
    ('scaler', StandardScaler())
])

onehot_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True)),
    ('onehot', OneHotEncoder(sparse=False, drop='first')) # Using 'drop' to avoid multicollinearity
])

ordinal_pipeline = Pipeline([
    ('ordinal', OrdinalEncoder(
        categories=[cat for cat in ordinal_map.values()],
        unknown_value=np.nan,
        handle_unknown='use_encoded_value',
    ))
])

preprocessor = ColumnTransformer([
    ('numeric', numerical_pipeline, numerical),
    ('frontage', frontage_pipeline, ['LotFrontage']),
    ('onehot', onehot_pipeline, onehot),
    ('ordinal', ordinal_pipeline, ordinal)
])

In [19]:
X_tf = preprocessor.fit_transform(X_concat)

In [20]:
num_of_row_in_train = X.shape[0]
X_train = pd.DataFrame(X_tf[:num_of_row_in_train, :])
X_test = pd.DataFrame(X_tf[num_of_row_in_train:, :])

In [22]:
X_train.isna().sum().sort_values(ascending=False).head(11)

213    1453
221    1179
209     690
212      81
220      81
211      81
217      38
215      38
207      37
216      37
208      37
dtype: int64

Looking at final data and export it!

In [23]:
train = pd.concat([X_train, y_train], axis=1)
train

0         1         2         3         4         5         6  \
0    -0.202068 -0.217879  0.646183 -0.507284  1.046258  0.896833  0.525202   
1     0.501870 -0.072044 -0.063185  2.188279  0.154764 -0.395604 -0.572250   
2    -0.061280  0.137197  0.646183 -0.507284  0.980221  0.848965  0.334828   
3    -0.436714 -0.078385  0.646183 -0.507284 -1.859351 -0.682812 -0.572250   
4     0.689587  0.518903  1.355551 -0.507284  0.947203  0.753229  1.387486   
...        ...       ...       ...       ...       ...       ...       ...   
1455 -0.342855 -0.285470 -0.063185 -0.507284  0.914184  0.753229 -0.572250   
1456  0.736516  0.381311 -0.063185  0.391237  0.220801  0.178812  0.094060   
1457 -0.155138 -0.142806  0.646183  3.086800 -1.000876  1.040437 -0.572250   
1458 -0.061280 -0.057207 -0.772552  0.391237 -0.703711  0.561757 -0.572250   
1459  0.267224 -0.029308 -0.772552  0.391237 -0.208437 -0.922153 -0.572250   

             7         8         9        10        11        12        13  \
0     0.580907 -0.293130 -0.934863 -0.444328 -0.773861  1.207379 -0.101197   
1     1.178112 -0.293130 -0.629896  0.477111  0.261075 -0.785025 -0.101197   
2     0.097873 -0.293130 -0.288516 -0.299076 -0.610718  1.235375 -0.101197   
3    -0.494941 -0.293130 -0.047275 -0.671283 -0.506205  0.978742 -0.101197   
4     0.468931 -0.293130 -0.161068  0.211573 -0.037170  1.671651 -0.101197   
...        ...       ...       ...       ...       ...       ...       ...   
1455 -0.969192 -0.293130  0.892658 -0.224181 -0.526598  0.834095 -0.101197   
1456  0.765338  0.670525  0.064243  1.112586  2.328397 -0.785025 -0.101197   
1457 -0.365400 -0.293130  0.719693  0.227460  0.072441  1.902620 -0.101197   
1458 -0.861608  5.790313 -1.276243  0.059513 -0.207960 -0.785025 -0.101197   
1459  0.853162  1.421349 -0.966725  0.463494  0.245780 -0.785025 -0.101197   

            14        15        16        17        18        19        20  \
0     0.413547  1.087023 -0.249895  0.781366  1.232599  0.169927 -0.207698   
1    -0.471891 -0.819679  3.822419  0.781366 -0.756321  0.169927 -0.207698   
2     0.563755  1.087023 -0.249895  0.781366  1.232599  0.169927 -0.207698   
3     0.427382  1.087023 -0.249895 -1.027363 -0.756321  0.169927 -0.207698   
4     1.378042  1.087023 -0.249895  0.781366  1.232599  1.385655 -0.207698   
...        ...       ...       ...       ...       ...       ...       ...   
1455  0.289033 -0.819679 -0.249895  0.781366  1.232599  0.169927 -0.207698   
1456  1.130989  1.087023 -0.249895  0.781366 -0.756321  0.169927 -0.207698   
1457  1.658694 -0.819679 -0.249895  0.781366 -0.756321  1.385655 -0.207698   
1458 -0.835553  1.087023 -0.249895 -1.027363 -0.756321 -1.045801 -0.207698   
1459 -0.483749  1.087023 -0.249895 -1.027363  1.232599  0.169927 -0.207698   

            21        22        23        24        25        26        27  \
0     0.986849 -0.924311  1.000929  0.306528  0.348900 -0.740760  0.200006   
1    -0.287758  0.623632 -0.085000  0.306528 -0.059792  1.614879 -0.702843   
2    -0.287758  0.623632  0.920490  0.306528  0.627553 -0.740760 -0.081209   
3     0.349546  0.623632  0.799831  1.619961  0.785457 -0.740760 -0.184815   
4     1.624153  0.623632  0.880270  1.619961  1.686437  0.776967  0.540424   
...        ...       ...       ...       ...       ...       ...       ...   
1455  0.349546  0.623632  0.840050  0.306528 -0.059792 -0.740760 -0.110811   
1456  0.349546  2.171575 -0.004561  0.306528  0.125977  2.018026 -0.702843   
1457  1.624153  2.171575 -1.492686 -1.006906 -1.025792 -0.740760  0.185205   
1458 -0.925062 -0.924311 -1.130710 -1.006906 -1.081523  2.152408 -0.702843   
1459 -0.287758 -0.924311 -0.527416 -1.006906 -0.914331  5.077195  0.303612   

            28        29        30       31        32        33        34  \
0    -0.359601 -0.103331 -0.285935 -0.06315 -0.089592 -1.552184  0.157646   
1    -0.359601 -0.103331 -0.285935 -0.06315 -0.089592 -0.446925 -0.602962   
2    -0.359601 -0.103331 -0.285935

In [24]:
test = X_test
test

0         1         2         3         4         5         6    \
0     0.501870  0.184371 -0.772552  0.391237 -0.340510 -1.113625 -0.572250   
1     0.548800  0.519791 -0.063185  0.391237 -0.439565 -1.257229  0.032468   
2     0.220295  0.464374 -0.772552 -0.507284  0.848148  0.657493 -0.572250   
3     0.408012 -0.024109 -0.063185  0.391237  0.881166  0.657493 -0.460265   
4    -1.234510 -0.654748  1.355551 -0.507284  0.683057  0.370284 -0.572250   
...        ...       ...       ...       ...       ...       ...       ...   
1454 -2.266952 -1.043937 -1.481920  1.289758 -0.043346 -0.682812 -0.572250   
1455 -2.266952 -1.049263 -1.481920 -0.507284 -0.043346 -0.682812 -0.572250   
1456  4.256207  1.246808 -0.772552  1.289758 -0.373528  0.561757 -0.572250   
1457 -0.342855  0.034605 -0.772552 -0.507284  0.683057  0.370284 -0.572250   
1458  0.220295 -0.068620  0.646183 -0.507284  0.716075  0.466021 -0.045921   

           7         8         9         10        11        12        13   \
0     0.058352  0.558197 -0.661759 -0.385319 -0.671897 -0.785025 -0.101197   
1     1.057354 -0.293130 -0.352241  0.629171  0.431865 -0.785025 -0.101197   
2     0.767534 -0.293130 -0.964449 -0.280920 -0.590326  0.850426 -0.101197   
3     0.352564 -0.293130 -0.538862 -0.285459 -0.595424  0.796766 -0.101197   
4    -0.391747 -0.293130  1.038314  0.517963  0.306959 -0.785025 -0.101197   
...        ...       ...       ...       ...       ...       ...       ...   
1454 -0.969192 -0.293130 -0.033619 -1.147889 -1.564083  0.488807 -0.101197   
1455 -0.415899 -0.293130 -0.607138 -1.147889 -1.564083  0.488807 -0.101197   
1456  1.718232 -0.293130 -1.276243  0.390868  0.164209 -0.785025 -0.101197   
1457 -0.229272 -0.293130  0.032381 -0.317233 -0.483263 -0.785025 -0.101197   
1458  0.695078 -0.293130 -0.734586 -0.126590 -0.416987  1.557332 -0.101197   

           14        15        16        17        18        19        20   \
0    -1.195262 -0.819679 -0.249895 -1.027363 -0.756321 -1.045801 -0.207698   
1    -0.339470 -0.819679 -0.249895 -1.027363  1.232599  0.169927 -0.207698   
2     0.253457 -0.819679 -0.249895  0.781366  1.232599  0.169927 -0.207698   
3     0.204046 -0.819679 -0.249895  0.781366  1.232599  0.169927 -0.207698   
4    -0.436315 -0.819679 -0.249895  0.781366 -0.756321 -1.045801 -0.207698   
...        ...       ...       ...       ...       ...       ...       ...   
1454 -0.807883 -0.819679 -0.249895 -1.027363  1.232599  0.169927 -0.207698   
1455 -0.807883 -0.819679 -0.249895 -1.027363  1.232599  0.169927 -0.207698   
1456 -0.546995  1.087023 -0.249895 -1.027363 -0.756321  1.385655 -0.207698   
1457 -1.049006 -0.819679  3.822419 -1.027363 -0.756321  0.169927 -0.207698   
1458  0.986710 -0.819679 -0.249895  0.781366  1.232599  0.169927 -0.207698   

           21        22        23        24        25        26        27   \
0    -0.925062 -0.924311 -0.688294 -1.006906  1.194149  0.365916 -0.702843   
1    -0.287758 -0.924311 -0.808953 -1.006906 -0.747138  2.365838 -0.170014   
2    -0.287758  0.623632  0.759611  0.306528  0.042381  0.935064 -0.199616   
3     0.349546  0.623632  0.799831  0.306528 -0.013350  2.104979 -0.170014   
4    -0.925062 -0.924311  0.558513  0.306528  0.153842 -0.740760  0.510823   
...        ...       ...       ...       ...       ...       ...       ...   
1454 -0.925062 -0.924311  0.000000 -2.320339 -2.196138 -0.740760 -0.702843   
1455 -0.287758 -0.924311 -0.326318 -1.006906 -0.867888 -0.740760 -0.347624   
1456  0.349546  0.623632 -0.728514  0.306528  0.478938  3.006130 -0.702843   
1457 -0.287758 -0.924311  0.000000 -2.320339 -2.196138 -0.108374 -0.229217   
1458  1.624153  0.623632  0.598733  1.619961  0.822611  0.761158  0.007596   

           28        29        30       31         32        33        34   \
0    -0.359601 -0.103331  1.850256 -0.06315  -0.089592 -0.078505  1.678860   
1    -0.359601 -0.103331 -0.285935 -0.06315  21.944410 -0.078505  1.678860   
2    -0.359601 -0.103331 -0.2

In [25]:
# train.to_csv("Datasets/train_le_ohe_oe.csv", index=False)
# test.to_csv("Datasets/test_le_ohe_oe.csv", index=False)